In [22]:
from PIL import Image
from google.colab import files
import os

DELIMITER = "####END####"
MIN_PANJANG_PESAN = 40

def teks_ke_bit(teks):
    bit_string = ""
    for huruf in teks:
        kode_ascii = ord(huruf)
        bit8 = f"{kode_ascii:08b}"
        bit_string += bit8
    return bit_string

def bit_ke_teks(bit_string):
    teks = ""
    for i in range(0, len(bit_string), 8):
        potongan = bit_string[i:i+8]
        if len(potongan) < 8:
            break
        angka = int(potongan, 2)
        teks += chr(angka)
    return teks

def sisipkan_pesan(gambar_masuk, gambar_keluar, pesan):
    pesan_lengkap = pesan + DELIMITER
    bit_pesan = teks_ke_bit(pesan_lengkap)

    img = Image.open(gambar_masuk)
    img = img.convert("RGB")
    piksel = img.load()

    lebar, tinggi = img.size
    kapasitas_bit = lebar * tinggi * 3

    if len(bit_pesan) > kapasitas_bit:
        raise ValueError(
            f"Pesan terlalu panjang: {len(bit_pesan)} bit, "
            f"kapasitas gambar hanya {kapasitas_bit} bit"
        )

    indeks_bit = 0
    for y in range(tinggi):
        for x in range(lebar):
            if indeks_bit >= len(bit_pesan):
                break

            r, g, b = piksel[x, y]
            channel_baru = []
            for channel in (r, g, b):
                if indeks_bit < len(bit_pesan):
                    bit = int(bit_pesan[indeks_bit])
                    channel = (channel & ~1) | bit
                    indeks_bit += 1
                channel_baru.append(channel)

            piksel[x, y] = tuple(channel_baru)

        if indeks_bit >= len(bit_pesan):
            break

    img.save(gambar_keluar)
    print("Pesan berhasil disisipkan ke:", gambar_keluar)
    print("Total bit pesan:", len(bit_pesan))
    print("Kapasitas gambar:", kapasitas_bit)

def ambil_pesan(gambar_stego):
    img = Image.open(gambar_stego)
    img = img.convert("RGB")
    piksel = img.load()

    lebar, tinggi = img.size
    bit_terkumpul = ""

    for y in range(tinggi):
        for x in range(lebar):
            r, g, b = piksel[x, y]
            bit_terkumpul += str(r & 1)
            bit_terkumpul += str(g & 1)
            bit_terkumpul += str(b & 1)

    teks_penuh = bit_ke_teks(bit_terkumpul)
    posisi = teks_penuh.find(DELIMITER)
    if posisi == -1:
        raise ValueError("Delimiter tidak ditemukan. Pesan mungkin tidak valid.")

    return teks_penuh[:posisi]


In [15]:
uploaded = files.upload()

list(uploaded.keys())

Saving ഒ.jpg to ഒ.jpg


['ഒ.jpg']

In [20]:
nama_gambar_asli = list(uploaded.keys())[0]
nama_gambar_stego = "stego_output.png"

pesan_rahasia = (
    "Tunggu denting terakhir dari lonceng kuno, "
    "saat kabut pagi menyentuh kusen jendela. "
    "Ingat sumpah yang tertulis di lipatan peta lama Gema-Di-Padang. "
    "Hanya kau yang tahu arti 'rumah kedua' itu."
    "Jangan tinggalkan jejak, kecuali debu waktu."
)

print("Panjang pesan:", len(pesan_rahasia))
if len(pesan_rahasia) < MIN_PANJANG_PESAN:
    raise ValueError("Pesan harus minimal 40 karakter!")

sisipkan_pesan(nama_gambar_asli, nama_gambar_stego, pesan_rahasia)

files.download(nama_gambar_stego)


Panjang pesan: 235
Pesan berhasil disisipkan ke: stego_output.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
pesan_diekstrak = ambil_pesan(nama_gambar_stego)
print("Pesan hasil ekstraksi:")
print(pesan_diekstrak)

print("Sama dengan pesan asli?", pesan_diekstrak == pesan_rahasia)


Pesan hasil ekstraksi:
Tunggu denting terakhir dari lonceng kuno, saat kabut pagi menyentuh kusen jendela. Ingat sumpah yang tertulis di lipatan peta lama Gema-Di-Padang. Hanya kau yang tahu arti 'rumah kedua' itu.Jangan tinggalkan jejak, kecuali debu waktu.
Sama dengan pesan asli? True
